# Vieira on GQA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vieira-artifact/vieira-artifact-aaai24/blob/main/gqa_main.ipynb)

In this notebook we explore using Vieira to solve the GQA visual question answer dataset.
We are going to run our Vieira program on a small dataset of 500 data-points randomly sampled from the original GQA dataset.
For simplicity of rerunning the dataset, we have prepared a dataset already containing pre-generated query programs using GPT-4.
For a full view of the program, please refer to the other GQA file in our repository.

In [1]:
# Checking python version
!python --version

Python 3.10.12


# Download our mini dataset

The dataset contains 500 samples

In [ ]:
# Dataset
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/dataset/gqa_mini_data.zip
!unzip gqa_mini_data.zip

# Download Vieira

In [3]:
# Download and install Vieira
# The default python version is for 3.10, you may change the link according to your python versions.
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira-0.2.2-cp310-cp310-manylinux_2_31_x86_64.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_ext-0.2.2-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_gpu-0.0.1-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_gpt-0.0.1-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_transformers-0.0.1-py3-none-any.whl
!wget https://github.com/vieira-artifact/vieira-artifact-aaai24/releases/download/v0.2.2/vieira_opencv-0.0.1-py3-none-any.whl
!pip install vieira-0.2.2-cp310-cp310-manylinux_2_31_x86_64.whl
!pip install vieira_ext-0.2.2-py3-none-any.whl
!pip install vieira_gpu-0.0.1-py3-none-any.whl
!pip install vieira_gpt-0.0.1-py3-none-any.whl
!pip install vieira_transformers-0.0.1-py3-none-any.whl
!pip install vieira_opencv-0.0.1-py3-none-any.whl

Processing ./vieira_opencv-0.0.1-py3-none-any.whl


# Import Vieira!

In [1]:
# Import vieira and related plugins
import vieira
import vieira_ext

# Setup Vieira plugins.

In this application, GPU, GPT, Transformers, and OpenCV plugins will be enabled.

In [2]:
# Configure Vieira plugins
import argparse
plugins = vieira_ext.PluginRegistry()

parser = argparse.ArgumentParser()
plugins.setup_argument_parser(parser)
known_args, unknown_args = parser.parse_known_args()
plugins.configure(known_args, unknown_args)

[vieira_openai] `OPENAI_API_KEY` not found, consider setting it in the environment variable


# Code loading the dataset and GPT-4 generated programs.

In [3]:
# Get dataset
import pickle
import io
import tokenize

GQA_PATH = '/content/gqa_mini_data/'

# Used to parse each data point's input program
class Program:
  def __init__(self, prog_str, init_state=None):
    self.prog_str = prog_str
    self.state = init_state if init_state is not None else dict()
    self.instructions = self.prog_str.split('\n')
    self.progs = [parse_step(i) for i in self.instructions]

  def __repr__(self):
    return self.prog_str

def parse_step(step_str, partial=False):
  tokens = list(tokenize.generate_tokens(io.StringIO(step_str).readline))
  output_var = tokens[0].string
  step_name = tokens[2].string
  parsed_result = dict(
    output_var=output_var,
    step_name=step_name)
  if partial:
    return parsed_result

  arg_tokens = [token for token in tokens[4:-3] if token.string not in [',','=']]
  num_tokens = len(arg_tokens) // 2
  args = dict()
  for i in range(num_tokens):
    args[arg_tokens[2*i].string] = arg_tokens[2*i+1].string
  parsed_result['args'] = args
  return parsed_result

def get_dataset():
  with open(GQA_PATH + 'mini_question_no_crop.pkl', 'rb') as f:
    return pickle.load(f)

# The Vieira Program

In [4]:
# Create Vieira context with Vieira code
def create_context():
  ctx = vieira.Context(provenance="topkproofs")
  plugins.load_into_ctx(ctx)
  ctx.set_iter_limit(100)

  ctx.add_program("""
  @owl_vit(output_fields=["bbox-x", "bbox-y", "bbox-w", "bbox-h"], limit=5)
  type find_object(bound img: Tensor, bound object: String, id: u32, x: u32, y: u32, w: u32, h: u32)

  @vilt(top=1)
  type vqa(bound img: Tensor, bound question: String, answer: String)

  @py_eval
  type $py_eval_string(s: String) -> String

  // all args are variable names
  type Expr = IMAGE(String)
            | CROP(String, String)
            | CROP_ABOVE(String, String)
            | CROP_BELOW(String, String)
            | CROP_LEFTOF(String, String)
            | CROP_RIGHTOF(String, String)
            | LOC(String, String)
            | VQA(String, String)
            | COUNT(String)
            | EVAL(String)
            | RESULT(String)

  type process_eval_string(bound id: u32, bound str: String, processed: String)
  rel process_eval_string(n, s, s) = n == 0
  rel process_eval_string(n + 1, str, $string_replace(str_to_eval, $format("{{}}", var), val as String)) = var_value_int(var, val, n) and process_eval_string(n, str, str_to_eval)
  rel process_eval_string(n + 1, str, $string_replace(str_to_eval, $format("{{}}", var), val as String)) = var_value_string(var, val, n) and process_eval_string(n, str, str_to_eval)
  rel process_eval_string(n + 1, str, str_to_eval) = step(n, _, e) and case e is LOC(_, _) and process_eval_string(n, str, str_to_eval)
  rel process_eval_string(n + 1, str, str_to_eval) = var_value_tensor(_, _, n) and process_eval_string(n, str, str_to_eval)

  rel eval_image(e, $load_image(img_path)) = case e is IMAGE(img_path)
  rel eval_image(e, $crop_image(img, x, y, w, h)) = case e is CROP(img_var, box_var) and var_value_tensor(img_var, img, _) and var_value_bbox(box_var, _, x, y, w, h, _)
  rel eval_image(e, $crop_image(img, x, y, w, h, "above")) = case e is CROP_ABOVE(img_var, box_var) and var_value_tensor(img_var, img, _) and var_value_bbox(box_var, _, x, y, w, h, _)
  rel eval_image(e, $crop_image(img, x, y, w, h, "below")) = case e is CROP_BELOW(img_var, box_var) and var_value_tensor(img_var, img, _) and var_value_bbox(box_var, _, x, y, w, h, _)
  rel eval_image(e, $crop_image(img, x, y, w, h, "left")) = case e is CROP_LEFTOF(img_var, box_var) and var_value_tensor(img_var, img, _) and var_value_bbox(box_var, _, x, y, w, h, _)
  rel eval_image(e, $crop_image(img, x, y, w, h, "right")) = case e is CROP_RIGHTOF(img_var, box_var) and var_value_tensor(img_var, img, _) and var_value_bbox(box_var, _, x, y, w, h, _)
  rel eval_bbox(e, id, x, y, w, h) = case e is LOC(img_name, object) and var_value_tensor(img_name, image, _) and find_object(image, object, id, x, y, w, h)

  type eval_string(bound id: u32, bound e: Expr, s: String)
  rel eval_string(id, e, answer) = case e is VQA(img_name, question) and var_value_tensor(img_name, image, _) and vqa(image, question, answer)
  rel eval_string(id, e, $py_eval_string(str_to_eval)) = case e is EVAL(str) and process_eval_string(id, str, str_to_eval)
  rel eval_string(id, e, result) = case e is RESULT(var_name) and var_value_string(var_name, result, _)
  rel eval_string(id, e, result as String) = case e is RESULT(var_name) and var_value_int(var_name, result, _)

  type eval_int(e: Expr, n: usize)
  rel eval_int(e, cnt) = cnt := count(bid: var_value_bbox(box_name, bid, _, _, _, _, _) where e: case e is COUNT(box_name))

  rel var_value_tensor(var_name, image, id) = step(id, var_name, expr) and eval_image(expr, image)
  rel var_value_bbox(var_name, id, x, y, w, h, sid) = step(sid, var_name, expr) and eval_bbox(expr, id, x, y, w, h)
  rel var_value_string(var_name, str, id) = step(id, var_name, expr) and eval_string(id, expr, str)
  rel var_value_int(var_name, val, id) = step(id, var_name, expr) and eval_int(expr, val)

  rel final_result(answer) = var_value_string("FINAL_RESULT", answer, _)

  query final_result
  """)

  ctx.set_non_probabilistic("step")
  return ctx

# Testing Scripts

In [7]:
from tqdm import tqdm

# For parsing program input
ARG_ORDER = {
  "COUNT": ("box",),
  "CROP": ("image", "box"),
  "CROP_ABOVE": ("image", "box"),
  "CROP_BELOW": ("image", "box"),
  "CROP_LEFTOF": ("image", "box"),
  "CROP_RIGHTOF": ("image", "box"),
  "EVAL": ("expr",),
  "LOC": ("image", "object"),
  "RESULT": ("var",),
  "VQA": ("image", "question"),
}

# Accuracy is evaluated with string matching @ best k predictions
def check_prediction(predictions, ground_truth, k):
  return any(ground_truth in pred or pred in ground_truth for pred in predictions[:k])

# Test a single data point
def test_one(context, id, testcase):
  image_path = GQA_PATH + "images/" + testcase["imageId"] + ".jpg"   # FIX IMAGE PATH
  step_facts = [(0, "IMAGE", f'IMAGE("{image_path}")')]

  for i, var_dict in enumerate(testcase["prog"].progs):
    function = var_dict["step_name"]
    if function not in ARG_ORDER:
      continue
    arg_str_list = []
    for arg in ARG_ORDER[function]:
      if arg == "expr":
        arg_str_list.append(var_dict["args"][arg])
      elif arg == "object" or arg == "question":
        arg_str_list.append(var_dict["args"][arg].replace("'", '"'))
      else:
        arg_str_list.append('"' + var_dict["args"][arg] + '"')
    expr_str = function + "(" + ",".join(arg_str_list) + ")"
    step_facts.append((i + 1, var_dict["output_var"], expr_str))

  ctx = context.clone()
  ctx.add_facts("step", step_facts)
  ctx.run()
  result = list(ctx.relation("final_result"))

  if result:
    result.sort(key=lambda x: x[0], reverse=True)
    return [tup[1][0] for tup in result[:5]]
  else:
    return ["no"]

# Running the Experiment!

Please checkout the log to get details of experimental results

In [8]:
# Run experiment on the dataset
context = create_context()

data = get_dataset()
items = tqdm(list(data.items()))
results = {}
correct = {1: 0, 3: 0, 5: 0}
match_substring = lambda s1, s2: s1 in s2 or s2 in s1
total = 0
for id, testcase in items:
  ground_truth = testcase["answer"]
  predictions = test_one(context, id, testcase)
  results[id] = predictions + [ground_truth]

  for k in (1, 3, 5):
    if check_prediction(predictions, ground_truth, k):
      correct[k] += 1
  total += 1

  print(f"ground truth: {ground_truth}, predictions: {predictions}")
  print(f"total: {total}, correct: {correct}")
  print(testcase["prog"].prog_str)

print(total)
print(correct)
print(results)

 62%|██████▏   | 310/501 [48:06<33:54, 10.65s/it]

ground truth: no, predictions: ['yes']
total: 310, correct: {1: 182, 3: 202, 5: 203}
BOX0=LOC(image=IMAGE,object='bird')
IMAGE0=CROP_LEFTOF(image=IMAGE,box=BOX0)
BOX1=LOC(image=IMAGE0,object='young elephant')
ANSWER0=COUNT(box=BOX1)
ANSWER1=EVAL(expr="'yes' if {ANSWER0} > 0 else 'no'")
FINAL_RESULT=RESULT(var=ANSWER1)


 62%|██████▏   | 311/501 [48:07<24:24,  7.71s/it]

ground truth: no, predictions: ['no']
total: 311, correct: {1: 183, 3: 203, 5: 204}
ANSWER0=VQA(image=IMAGE,question='Is that man running?')
FINAL_RESULT=RESULT(var=ANSWER0)


 62%|██████▏   | 312/501 [51:41<3:39:25, 69.66s/it]

...

ground truth: yes, predictions: ['yes']
total: 499, correct: {1: 289, 3: 331, 5: 332}
BOX0=LOC(image=IMAGE,object='hydrant')
IMAGE0=CROP_RIGHTOF(image=IMAGE,box=BOX0)
BOX1=LOC(image=IMAGE0,object='person')
IMAGE1=CROP(image=IMAGE0,box=BOX1)
ANSWER0=VQA(image=IMAGE1,question='Is the person wearing a jacket?')
FINAL_RESULT=RESULT(var=ANSWER0)


100%|█████████▉| 500/501 [1:25:45<00:27, 27.18s/it]

ground truth: yes, predictions: ['no', 'yes']
total: 500, correct: {1: 289, 3: 332, 5: 333}
BOX0=LOC(image=IMAGE,object='LEFT')
IMAGE0=CROP(image=IMAGE,box=BOX0)
BOX1=LOC(image=IMAGE0,object='picture')
ANSWER0=COUNT(box=BOX1)
ANSWER1=EVAL(expr="'yes' if {ANSWER0} > 0 else 'no'")
FINAL_RESULT=RESULT(var=ANSWER1)


100%|██████████| 501/501 [1:25:50<00:00, 10.28s/it]

ground truth: table, predictions: ['table']
total: 501, correct: {1: 290, 3: 333, 5: 334}
BOX0=LOC(image=IMAGE,object='furniture')
IMAGE0=CROP(image=IMAGE,box=BOX0)
ANSWER0=VQA(image=IMAGE0,question='What is this item of furniture called?')
FINAL_RESULT=RESULT(var=ANSWER0)
501
{1: 290, 3: 333, 5: 334}
{'19220961': ['chair', 'bench'], '07267508': ['table', 'bottle'], '15512494': ['no', 'no'], '00464293': ['no', 'yes', 'yes'], '10233633': ['no', 'yes', 'yes'], '1679555': ['no', 'yes'], '13852280': ['no', 'yes', 'no'], '151004785': ['modern', 'antique'], '01248326': ['yes', 'no', 'no'], '12976953': ['no', 'yes', 'no'], '17908046': ['no', 'yes', 'no'], '05712037': ['no', 'yes'], '05673874': ['brown', 'black', 'brown'], '06407231': ['yes', 'no', 'yes'], '12863527': ['no', 'no'], '04587867': ['short', 'long'], '1923285': ['no', 'yes'], '07794099': ['no', 'blue'], '01776009': ['no', 'yes'], '04711033': ['no', 'no'], '14596273': ['yes', 'no', 'yes'], '14892279': ['no', 'no'], '13389301': ['no'

# Performance

Here we report the score under Recall@$k$ metric, where $k = 1, 3, 5$.

In [10]:
# Print result
print("Recall@1:", correct[1] / total)
print("Recall@3:", correct[3] / total)
print("Recall@5:", correct[5] / total)

Recall@1: 0.5788423153692615
Recall@3: 0.6646706586826348
Recall@5: 0.6666666666666666
